In [18]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import cv2
from sklearn.model_selection import train_test_split


# Importing Deep Learning Libraries

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam,SGD,RMSprop

In [19]:
picture_size = 48
folder_path = "images/train"


In [20]:
def load_and_preprocess_images(folder_path, picture_size):
    images = []
    labels = []
    emotion_labels = sorted(os.listdir(folder_path))  # Lấy danh sách các thư mục (nhãn cảm xúc)

    for label in emotion_labels:
        label_path = os.path.join(folder_path, label)
        for image_file in os.listdir(label_path):
            image_path = os.path.join(label_path, image_file)
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Chuyển đổi ảnh sang màu xám
            image = cv2.resize(image, (picture_size, picture_size))  # Thay đổi kích thước ảnh
            image = image.astype('float32') / 255.0  # Chuẩn hóa ảnh
            images.append(image)
            labels.append(emotion_labels.index(label))  # Gán nhãn cảm xúc

    images = np.array(images)
    labels = np.array(labels)

    # Chuyển đổi hình ảnh thành định dạng phù hợp cho mô hình Keras
    images = np.expand_dims(images, -1)
    return images, labels

images, labels = load_and_preprocess_images(folder_path, picture_size)


In [21]:
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.2, random_state=42)
X_dev, X_test, y_dev, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [22]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    shear_range=0.2,
    fill_mode='nearest'
)

datagen.fit(X_train)



In [23]:
def apply_gaussian_blur(images):
    blurred_images = []
    for image in images:
        blurred_image = cv2.GaussianBlur(image, (5, 5), 0)
        blurred_images.append(blurred_image)
    return np.array(blurred_images)

X_train_blurred = apply_gaussian_blur(X_train)


In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(picture_size, picture_size, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(os.listdir(folder_path)), activation='softmax')  # Số lượng lớp đầu ra bằng số lượng nhãn cảm xúc
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_dev, y_dev),
                    epochs=30)


C:\Users\theng\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30


C:\Users\theng\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


721/721 ━━━━━━━━━━━━━━━━━━━━ 41s 52ms/step - accuracy: 0.2424 - loss: 1.8218 - val_accuracy: 0.3588 - val_loss: 1.6555
Epoch 2/30
721/721 ━━━━━━━━━━━━━━━━━━━━ 35s 48ms/step - accuracy: 0.3101 - loss: 1.7117 - val_accuracy: 0.3789 - val_loss: 1.6149
Epoch 3/30
721/721 ━━━━━━━━━━━━━━━━━━━━ 35s 48ms/step - accuracy: 0.3523 - loss: 1.6352 - val_accuracy: 0.4261 - val_loss: 1.5003
Epoch 4/30
721/721 ━━━━━━━━━━━━━━━━━━━━ 35s 48ms/step - accuracy: 0.3702 - loss: 1.6031 - val_accuracy: 0.4417 - val_loss: 1.4603
Epoch 5/30
721/721 ━━━━━━━━━━━━━━━━━━━━ 36s 49ms/step - accuracy: 0.3914 - loss: 1.5591 - val_accuracy: 0.4504 - val_loss: 1.4430
Epoch 6/30
721/721 ━━━━━━━━━━━━━━━━━━━━ 35s 49ms/step - accuracy: 0.4076 - loss: 1.5355 - val_accuracy: 0.4591 - val_loss: 1.4134
Epoch 7/30
721/721 ━━━━━━━━━━━━━━━━━━━━ 35s 49ms/step - accuracy: 0.4061 - loss: 1.5258 - val_accuracy: 0.4511 - val_loss: 1.4074
Epoch 8/30
721/721 ━━━━━━━━━━━━━━━━━━━━ 35s 48ms/step - accuracy: 0.4268 - loss: 1.4829 - val_accurac

In [25]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy}')


91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5467 - loss: 1.1964
Test accuracy: 0.539715588092804


In [26]:
# Lưu mô hình vào file .h5
model.save('emotion_model.h5')
